In [1]:
import pandas as pd

In [2]:
DIR = "/Users/damoncrockett/Desktop/astrophotography_data/"

In [3]:
df = pd.read_csv("./fuck.csv")

In [5]:
df.columns

Index([u'local_path', u'lat', u'lon', u'qual'], dtype='object')

In [6]:
from PIL import Image, ImageDraw
from shapely.geometry import Point
import os
import math
import numpy as np

In [7]:
tmp = df

In [8]:
var = "qual"

In [9]:
tmp = tmp[tmp.qual!=0]

In [10]:
#tmp = tmp[tmp.ISO!="65535+0+0"]

In [11]:
#tmp.focal = tmp.focal.apply(float)

In [12]:
tmp.sort(var,ascending=False,inplace=True)
tmp.reset_index(drop=True,inplace=True)

/Library/Python/2.7/site-packages/pandas/core/frame.py:2915: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  inplace=inplace, kind=kind, na_position=na_position)


In [13]:
tmp.head()

,local_path,lat,lon,qual
0,/Users/damoncrockett/Desktop/astrophotography_...,-34.252478,-59.391695,4
1,/Users/damoncrockett/Desktop/astrophotography_...,46.936491,6.719169,4
2,/Users/damoncrockett/Desktop/astrophotography_...,53.135766,-4.415176,4
3,/Users/damoncrockett/Desktop/astrophotography_...,34.027486,-84.590456,4
4,/Users/damoncrockett/Desktop/astrophotography_...,46.936491,6.719169,4


In [14]:
n = len(tmp)

grid_side = int(round( np.sqrt(n) / 0.75 ))

x,y = range(grid_side) * grid_side, np.repeat(range(grid_side),grid_side)
grid_list = pd.DataFrame(x,columns=['x'])
grid_list['y'] = y

point = []
for i in range(len(grid_list)):
    point.append(Point(grid_list.x.loc[i],grid_list.y.loc[i]))

grid_list['point'] = point

open_grid = list(grid_list.point)

midpoint = int(round( grid_side / 2 ))
centroid_loc = Point(midpoint,midpoint)

if n > 1:
    open_grid.remove(centroid_loc)

thumb_side = 128

px_w = thumb_side * grid_side
px_h = thumb_side * grid_side

canvas = Image.new('RGB',(px_w,px_h),(0,0,0))

for i in range(n):
    im = Image.open(tmp.local_path.loc[i])
    im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
    closest_open = min(open_grid,key=lambda x: centroid_loc.distance(x))
    x = int(closest_open.x) * thumb_side
    y = int(closest_open.y) * thumb_side

    #pos = (7,7)
    #draw = ImageDraw.Draw(im)
    #draw.text(pos, str(i),fill="white")

    canvas.paste(im,(x,y))
    open_grid.remove(closest_open)

canvas.save(DIR+"qual.png")